In [417]:
%reload_ext black
from bs4 import BeautifulSoup
from bs4 import NavigableString
import pandas as pd
import pprint
import requests
import datetime
import urllib

In [418]:
news_url = 'https://mars.nasa.gov/news/'
jpl_root = 'https://www.jpl.nasa.gov'
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# Latest Mars Articles

In [419]:
#open and parse the page
page = urllib.request.urlopen(news_url)
news_soup = BeautifulSoup(page, 'html.parser')

#zip title and description items for iterability
zipped = zip(
    news_soup.find_all('div', class_='content_title'),
    news_soup.find_all('div', class_='image_and_description_container')
            )

#loop though zip object and extract data
articles = []
for div in zipped:
    info_dict = {
    'article_title' : div[0].a.text.strip(),
    #'date_posted' : div[2].text.strip(),
    'description_text' : div[1].text.strip(),
    'article_link' : div[1].a['href'].strip(),
    }
    articles.append(info_dict)

##grab the latest article
latest_article_title = articles[0]['article_title']
latest_article_desc = articles[0]['description_text']

print(latest_article_title)
print(latest_article_desc)

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries.


# Mars Images from JPL

In [420]:
#open and parse the page
r = requests.get(img_url).text
img_soup = BeautifulSoup(r, 'html.parser')

latest_img_title = (img_soup.find('div', class_='carousel_items')
                    .div.h1.text.strip())
latest_img_url = str(img_soup.find('div', class_='carousel_items')
                     .div.footer.a.attrs['data-fancybox-href'])

latest_img_url_full = jpl_root + latest_img_url

print(latest_img_title)
print(latest_img_url_full)

Galactic Gathering Gives Impressive Light Display
https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18903_ip.jpg


In [424]:
twiter_url = 'https://twitter.com/marswxreport?lang=en'
r = requests.get(twiter_url).text
twit_soup = BeautifulSoup(r, 'html.parser')

tw_zip = zip(
    twit_soup.find_all('div', class_="js-tweet-text-container"), 
    twit_soup.find_all('span', class_="_timestamp js-short-timestamp")
            )
weather = []
for tweet in tw_zip:
    
    info_dict = {'tweet_text' : tweet[0].p.text,
                'date' : datetime.date.fromtimestamp(int(tweet[1].attrs['data-time']))}
    if info_dict['tweet_text'].startswith('InSight') and info_dict not in weather:
        weather.append(info_dict)
# div class="js-tweet-text-container">
# <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">InSight sol 451 (2020-03-03) low -93.6ºC (-136.4ºF) high -10.3ºC (13.4ºF)
# winds from the SSE at 6.3 m/s (14.2 mph) gusting to 19.9 m/s (44.5 mph)
# pressure at 6.30 hPa<a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/s9U5e7aC8b">pic.twitter.com/s9U5e7aC8b</a></p>
# </div>

    
# <a class="tweet-timestamp js-permalink js-nav js-tooltip" 
# data-conversation-id="1234899138204119040" 
# href="/MarsWxReport/status/1234899138204119040" 
# title="9:51 AM - 3 Mar 2020">
# <span class="_timestamp js-short-timestamp" 
# data-aria-label-part="last" data-long-form="true"
# data-time="1583257872" data-time-ms="1583257872000">Mar 3</span></a>

In [425]:
weather

[{'tweet_text': 'InSight sol 457 (2020-03-10) low -95.7ºC (-140.3ºF) high -9.1ºC (15.6ºF)\nwinds from the SSE at 6.5 m/s (14.5 mph) gusting to 21.0 m/s (46.9 mph)\npressure at 6.30 hPapic.twitter.com/2h0LKoSQrJ',
  'date': datetime.date(2020, 3, 8)},
 {'tweet_text': 'InSight sol 456 (2020-03-08) low -94.6ºC (-138.3ºF) high -9.6ºC (14.7ºF)\nwinds from the SSE at 5.8 m/s (12.9 mph) gusting to 20.2 m/s (45.2 mph)\npressure at 6.30 hPa',
  'date': datetime.date(2020, 3, 7)},
 {'tweet_text': 'InSight sol 455 (2020-03-08) low -95.4ºC (-139.8ºF) high -13.0ºC (8.5ºF)\nwinds from the SSE at 6.0 m/s (13.5 mph) gusting to 20.7 m/s (46.2 mph)\npressure at 6.40 hPapic.twitter.com/HrRcPTpSEQ',
  'date': datetime.date(2020, 3, 5)},
 {'tweet_text': 'InSight sol 454 (2020-03-06) low -96.0ºC (-140.8ºF) high -12.7ºC (9.1ºF)\nwinds from the SSE at 5.8 m/s (12.9 mph) gusting to 25.2 m/s (56.4 mph)\npressure at 6.30 hPapic.twitter.com/SW2gBfJY3o',
  'date': datetime.date(2020, 3, 5)},
 {'tweet_text': 'InSig